# Start the Project 🤩


## Step One 
### Exploratory Data Analysis
**Before Data Preprocessing**

-  Image Count & Class Distribution

-  Random Image Visualization

-  Image Shape / Size / Channels Check

- File Format Check (.jpg / .png / etc.)

- Missing / Corrupt / Blank Images